In [4]:
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias


df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features.features_de_csvs(df_train, df_eval)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_eval_idf = pd.read_csv('./data/test_idf.csv')

df_train = pd.merge(df_train, df_train_idf, on= 'id', how= 'left')
df_eval = pd.merge(df_eval, df_eval_idf, on= 'id', how= 'left')

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.25)

df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)
df_train = features.llenar_nulls(df_train, hgb_mean=True)

In [5]:
# df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

# df_train_f = df_train_f.sample(frac=1).reset_index(drop=True)

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

## GridSearchCV

In [22]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV

features = ['habitaciones', 'garages','banos','antiguedad', 'metroscubiertos',  'metrostotales','lat_norm', 
           'lng_norm', 'gimnasio', 'usosmultiples', 'piscina']

features_test = ['top_provincia', 'promedio_precio_ciudad', 'anio', 'promedio_id_zona', 
                 'promedio_precio_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
               'count_tipo_propiedad_ciudad', 'promedio_precio_tipo_propiedad_ciudad_gen','count_id_zona'
           'dias_desde_datos','meses_desde_datos','porcentaje_metros','distancia_ciudad_centrica', 'puntaje', 'distancia_centro_mexico']

features += features_test

forest_params = { 
    "n_estimators": [10],
    "max_features": ["auto", "sqrt", "log2"],
    "min_samples_split": [2,4,8],
    "bootstrap": [True, False]
}

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=1)

rs_cv = GridSearchCV(estimator=RandomForestRegressor(), 
                           param_grid=forest_params, 
                           cv=4, scoring='neg_mean_absolute_error', verbose=1)

rs_cv.fit(x_train, y_train)

Fitting 4 folds for each of 18 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 14.1min finished


GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'bootstrap': [True, False],
   

In [24]:
display(rs_cv.best_params_)

{'bootstrap': False,
 'max_features': 'sqrt',
 'min_samples_split': 4,
 'n_estimators': 10}

In [30]:
from sklearn.ensemble import RandomForestRegressor


params = {'bootstrap': False,
 'max_features': 'sqrt',
 'min_samples_split': 4}

forest = RandomForestRegressor(verbose=1, n_estimators=100, **params)

features = ['habitaciones', 'garages','banos','antiguedad', 'metroscubiertos',  'metrostotales','lat_norm', 
           'lng_norm', 'gimnasio', 'usosmultiples', 'piscina']

features_test = ['top_provincia', 'promedio_precio_ciudad', 'anio', 'promedio_id_zona', 
                 'promedio_precio_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
               'count_tipo_propiedad_ciudad', 'promedio_precio_tipo_propiedad_ciudad_gen','count_id_zona'
           'dias_desde_datos','meses_desde_datos','porcentaje_metros','distancia_ciudad_centrica', 
                'distancia_centro_mexico', 'distancia_ciudad_cara']

features += features_test

x_train = utils.filtrar_features(df_train_f, features)
y_train = df_train_f['precio']
x_eval = utils.filtrar_features(df_test_f, features)
y_eval = df_test_f['precio']

forest.fit(x_train, y_train)

y_pred_train = forest.predict(x_train)
y_pred_eval = forest.predict(x_eval)

mae_train = utils.MAE(y_train, y_pred_train)
mae_eval = utils.MAE(y_eval, y_pred_eval)

print(f"MAE RandomForest (train): {mae_train:.5f}")
print(f"MAE RandomForest (eval): {mae_eval:.5f}")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    9.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


MAE RandomForest (train): 58145.90947
MAE RandomForest (eval): 506702.25115


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.0s finished
